In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten
from keras.callbacks import EarlyStopping

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
# Set the path to your data directory
data_directory = '/content/drive/My Drive/data'
# Function to calculate SDNN from heart rate data
def calculate_sdnn(heart_rate_values, window_size=5):
    # Convert heart rate to RR intervals (assuming constant heart rate for simplicity)
    rr_intervals = 60 / heart_rate_values  # since values are in bpm

    # Calculate SDNN over a moving window
    sdnn_values = []
    for i in range(len(rr_intervals) - window_size + 1):
        sdnn_values.append(np.std(rr_intervals[i:i+window_size]))

    return np.array(sdnn_values)

# Modified function to load and preprocess heart rate data
def load_and_preprocess_heartrate(subject_id):
    file_path = os.path.join(data_directory, 'heart_rate', f'{subject_id}_heartrate.txt')
    data = pd.read_csv(file_path, header=None, names=['date', 'heartrate'])
    heartrate_values = data['heartrate']

    # Calculate SDNN from heart rate data
    sdnn_values = calculate_sdnn(heartrate_values)

    return sdnn_values

def load_labeled_sleep(subject_id):
    file_path = os.path.join(data_directory, 'labels', f'{subject_id}_labeled_sleep.txt')
    data = pd.read_csv(file_path, header=None, delim_whitespace=True, names=['date', 'stage'])
    # Encode sleep labels as binary values: 1 for awake (label=0), 0 for asleep (labels 1-5)
    return (data['stage'] == 0).astype(int).values

subject_ids = [file.split('_')[0] for file in os.listdir(os.path.join(data_directory, 'labels')) if '_labeled_sleep.txt' in file]

X_heartrate = []
y_sleep = []

sequence_length = 400

for subject_id in subject_ids:
    heartrate_values = load_and_preprocess_heartrate(subject_id)
    sleep_labels = load_labeled_sleep(subject_id)

    for i in range(0, len(heartrate_values) - sequence_length):
        X_heartrate.append(heartrate_values[i:i+sequence_length])
        y_sleep.append(sleep_labels[min(i+sequence_length-1, len(sleep_labels)-1)])

In [14]:
X_heartrate = np.array(X_heartrate)
y_sleep = np.array(y_sleep)

label_encoder = LabelEncoder()
y_sleep = label_encoder.fit_transform(y_sleep)

scaler = StandardScaler()
X_heartrate_scaled = scaler.fit_transform(X_heartrate)

X_train, X_test, y_train, y_test = train_test_split(
    X_heartrate_scaled, y_sleep, test_size=0.1, random_state=42
)
# Model architecture
num_classes = 1  # Binary classification: awake or asleep

In [15]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape=(sequence_length,)),  # Input layer
    Dense(32, activation='relu'),  # Hidden layer
    Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

In [16]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping]  # Pass the early stopping callback
)
# sequence length 200: 79% accuracy
# sequence length 300: 0.813803518223746
# Model evaluation
y_pred = model.predict(X_test[..., np.newaxis])
y_pred_classes = (y_pred > 0.5).astype(int)  # Convert probabilities to binary classes
accuracy = accuracy_score(y_test, y_pred_classes)
print("Test Accuracy:", accuracy)

Epoch 1/20
6124/6124 [==============================] - 18s 3ms/step - loss: 0.5325 - accuracy: 0.7615 - val_loss: 0.5048 - val_accuracy: 0.7741
Epoch 2/20
6124/6124 [==============================] - 17s 3ms/step - loss: 0.4781 - accuracy: 0.7892 - val_loss: 0.4740 - val_accuracy: 0.7913
Epoch 3/20
6124/6124 [==============================] - 17s 3ms/step - loss: 0.4404 - accuracy: 0.8082 - val_loss: 0.4632 - val_accuracy: 0.7968
Epoch 4/20
6124/6124 [==============================] - 16s 3ms/step - loss: 0.4131 - accuracy: 0.8215 - val_loss: 0.4388 - val_accuracy: 0.8092
Epoch 5/20
6124/6124 [==============================] - 17s 3ms/step - loss: 0.3930 - accuracy: 0.8321 - val_loss: 0.4259 - val_accuracy: 0.8179
Epoch 6/20
6124/6124 [==============================] - 15s 3ms/step - loss: 0.3782 - accuracy: 0.8386 - val_loss: 0.4225 - val_accuracy: 0.8174
Epoch 7/20
6124/6124 [==============================] - 16s 3ms/step - loss: 0.3666 - accuracy: 0.8445 - val_loss: 0.4218 - val_ac